<a href="https://colab.research.google.com/github/Kane0068/classifying-documents/blob/main/Anahtar_%C5%9Eehir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Önce görüntü mü yoksa pdf mi ona karar verir
- Görüntüyse ona göre
* pdf ise ona göre okuma işlemi yapar
- İçinde tanımlanan şehir isimlerini tarar
- Bulursa uygun kategoriye ekler bulamazsa boş döndürür


In [ ]:
import cv2
import pytesseract
import PyPDF2
import re

def categorize_file(input_file, cities, category_folder):
    # PDF mi görüntü dosyası mı kontrol ediyoruz
    if input_file.lower().endswith(('.pdf', '.jpeg', '.jpg', '.png')):
        is_image = True
    else:
        is_image = False

    # Görüntü ise
    if is_image:
        image = cv2.imread(input_file)
        text = pytesseract.image_to_string(image)
    # Metin formatı ise
    else:
        pdf_file = open(input_file, "rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        text = ""
        for page_num in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
        pdf_file.close()

    # Şehir listesini yapalım
    matched_cities = [city for city in cities if re.search(city, text, re.I)]

    if matched_cities:
        # Kategoriyi belirle
        category = matched_cities[0]

        # Kategorinin dosya yolu
        category_path = os.path.join(category_folder, category)

        # Kategorinin klasörünü oluştur
        os.makedirs(category_path, exist_ok=True)

        # Metni ilgili kategoriye kaydet
        with open(os.path.join(category_path, os.path.basename(input_file)), "w") as file:
            file.write(text)

    return matched_cities

# Kullanım örneği:
input_file = "example.pdf"  # İşlem yapılacak dosyanın yolu
cities = ["Adana", "İstanbul", "Ankara"]  # İlgili şehirleri ekleyelim
category_folder = "categorized_data"  # Kategorilendirilmiş verilerin kaydedileceği klasör

matched_cities = categorize_file(input_file, cities, category_folder)
if matched_cities:
    print(f"Kategori: {matched_cities[0]}")


- TARAYICI KULLANARAK TARA
- TARANAN GÖRÜNTÜYÜ METNE ÇEVİR
- İŞLEMLERİ TEKRARLA

In [ ]:
import cv2
import pytesseract
import PyPDF2
import re
import shutil
import os
from scanner import Scanner  # Tarayıcı kütüphanesi

def scan_and_categorize(input_path, cities, category_folder):
    scanner = Scanner()  # Tarayıcı nesnesi oluşturun
    scanned_image = scanner.scan_to_file()  # Tarayıcıyı kullanarak bir görüntü alın
    input_file = input_path

    if not input_file.lower().endswith(('.pdf', '.jpeg', '.jpg', '.png')):
        raise ValueError("Desteklenmeyen dosya uzantısı.")

    # Görüntüyü metne çevir
    image = cv2.imread(scanned_image)
    text = pytesseract.image_to_string(image)

    # Şehir listesi
    matched_cities = [city for city in cities if re.search(city, text, re.I)]

    if matched_cities:
        # Kategoriyi belirle
        category = matched_cities[0]

        # Kategorinin dosya yolu
        category_path = os.path.join(category_folder, category)

        # Kategorinin klasörünü oluştur
        os.makedirs(category_path, exist_ok=True)

        # Metni ilgili kategoriye kaydet
        with open(os.path.join(category_path, os.path.basename(input_file)), "w") as file:
            file.write(text)

        # Tarayıcı tarafından kaydedilen geçici görüntüyü sil
        os.remove(scanned_image)

    return matched_cities

# Kullanım örneği:
input_path = "example.jpg"  # Tarayıcı ile tarama yapılacak dosyanın yolu
cities = ["Adana", "İstanbul", "Ankara"]  # İlgili şehirleri ekleyin
category_folder = "categorized_data"  # Kategorilendirilmiş verilerin kaydedileceği klasör

matched_cities = scan_and_categorize(input_path, cities, category_folder)
if matched_cities:
    print(f"Kategori: {matched_cities[0]}")


**-ÖNCEDEN EĞİTİLMİŞ CNN MODELİ ve OCR KULLANARAK SINIFLANDIRMA**

In [ ]:
# Görüntü verilerini işlemek için önceden eğitilmiş bir CNN modeli kullanın
import cv2
import pytesseract
from PIL import Image
import os

# Önceden eğitilmiş bir CNN modelini yükleyin (örneğin, ResNet-50)
cnn_model = tf.keras.applications.ResNet50(weights="imagenet")

# Görüntü dosyasını okuyun
image_path = "path/to/image.jpg"
image = cv2.imread(image_path)

# Görüntüyü önceden eğitilmiş model için uygun formata getirin
image = cv2.resize(image, (224, 224))  # Modelin giriş boyutuna uygun hale getirin
image = tf.keras.applications.resnet50.preprocess_input(image)
image = tf.convert_to_tensor(image)

# Görüntüyü model üzerinden geçirin ve sonucu alın
features = cnn_model.predict(tf.expand_dims(image, axis=0))

# OCR (Optical Character Recognition) kullanarak metni alın
text = pytesseract.image_to_string(Image.open(image_path))

# Metin içeriğine göre kategoriyi belirleyin
if "Adana" in text:
    category = "Adana"
else:
    category = "Diğer Şehir"

# Dosyaları kategorilere göre yerleştirin
output_dir = "output_folder"
category_dir = os.path.join(output_dir, category)
os.makedirs(category_dir, exist_ok=True)
output_file = os.path.join(category_dir, os.path.basename(image_path))
os.rename(image_path, output_file)


**FONKSİYON OLARAK DÜZENLENMİŞ HALİ**

In [ ]:
import cv2
import pytesseract
from PIL import Image
import os
import tensorflow as tf

def process_and_categorize_image(image_path, cnn_model, output_dir):
    # Görüntüyü önceden eğitilmiş model için uygun formata getirin
    def preprocess_image(image):
        image = cv2.resize(image, (224, 224))  # Modelin giriş boyutuna uygun hale getirin
        image = tf.keras.applications.resnet50.preprocess_input(image)
        image = tf.convert_to_tensor(image)
        return image

    # OCR (Optical Character Recognition) kullanarak metni alın
    def extract_text_from_image(image):
        text = pytesseract.image_to_string(Image.fromarray(image))
        return text

    # Dosyaları kategorilere göre yerleştirin
    def categorize_image(image_path, text, cnn_model, output_dir):
        features = cnn_model.predict(tf.expand_dims(preprocess_image(image), axis=0))

        if "Adana" in text:
            category = "Adana"
        else:
            category = "Diğer Şehir"

        category_dir = os.path.join(output_dir, category)
        os.makedirs(category_dir, exist_ok=True)
        output_file = os.path.join(category_dir, os.path.basename(image_path))
        os.rename(image_path, output_file)

    # Görüntü dosyasını okuyun
    image = cv2.imread(image_path)

    # Görüntüyü model üzerinden geçirin ve özellikleri alın
    categorize_image(image_path, extract_text_from_image(image), cnn_model, output_dir)

# Kullanım örneği
cnn_model = tf.keras.applications.ResNet50(weights="imagenet")
image_path = "path/to/image.jpg"
output_dir = "output_folder"
process_and_categorize_image(image_path, cnn_model, output_dir)


**HEM GÖRÜNTÜ HEM METİN TABANLI İÇİN**

In [ ]:
import cv2
import pytesseract
from PIL import Image
import os
import tensorflow as tf

def process_and_categorize_data(data_path, cnn_model, output_dir):
    def preprocess_image(image):
        image = cv2.resize(image, (224, 224))
        image = tf.keras.applications.resnet50.preprocess_input(image)
        image = tf.convert_to_tensor(image)
        return image

    def extract_text_from_image(image):
        text = pytesseract.image_to_string(Image.fromarray(image))
        return text

    def categorize_data(data_path, text, cnn_model, output_dir):
        features = cnn_model.predict(tf.expand_dims(preprocess_image(image), axis=0)

        if "Adana" in text:
            category = "Adana"
        else:
            category = "Diğer Şehir"

        category_dir = os.path.join(output_dir, category)
        os.makedirs(category_dir, exist_ok=True)
        output_file = os.path.join(category_dir, os.path.basename(data_path))
        os.rename(data_path, output_file)

    if data_path.endswith(('.jpg', '.jpeg', '.png')):
        image = cv2.imread(data_path)
        categorize_data(data_path, extract_text_from_image(image), cnn_model, output_dir)
    elif data_path.endswith('.pdf'):
        # ---------VVVVVVVVVVVV----------
        # PDF KODLARI YAZILACAK BURAYA
        # ---------^^^^^^^^^^^^-----------
        pass
    else:
        print("Desteklenmeyen dosya uzantısı")

# Kullanım örneği
cnn_model = tf.keras.applications.ResNet50(weights="imagenet")
data_path = "path/to/data.jpg"
output_dir = "output_folder"
process_and_categorize_data(data_path, cnn_model, output_dir)
